<a href="https://colab.research.google.com/github/VickkiMars/EagleLegal/blob/main/EagleRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp /content/drive/MyDrive/nigeria_laws_data.zip /content/

In [2]:
!unzip /content/nigeria_laws_data.zip

Archive:  /content/nigeria_laws_data.zip
   creating: nigeria_laws_data_20251107_112353/
  inflating: nigeria_laws_data_20251107_112353/processing_results.json  
   creating: nigeria_laws_data_20251107_112353/texts/
  inflating: nigeria_laws_data_20251107_112353/texts/_lawsofnigeria_view2.php.txt  
   creating: nigeria_laws_data_20251107_112353/pdfs/
  inflating: nigeria_laws_data_20251107_112353/pdfs/T8.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/U9.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/F7.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N1.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N40.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/C21.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/A23.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N29.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/N65.pdf  
  inflating: nigeria_laws_data_20251107_112353/pdfs/P42.pdf  
  inflating: nigeria_laws_data_

# Retrieve titles for text meteadata

In [3]:
# Install required packages
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium pandas tqdm

import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from tqdm import tqdm
from datetime import datetime

# Set up Chrome options for Colab
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-blink-features=AutomationControlled')
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Initialize the driver
driver = webdriver.Chrome(options=chrome_options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

def extract_links_with_text_from_page(page_number):
    """Extract URLs and their link text for links matching the two patterns"""
    try:
        url = f"https://placng.org/lawsofnigeria/?page={page_number}"
        print(f"Processing page {page_number}...")
        driver.get(url)

        # Wait for page to load
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "body"))
        )

        links_data = []

        # Find all <a> tags on the page
        all_links = driver.find_elements(By.TAG_NAME, "a")

        for link in all_links:
            href = link.get_attribute('href')
            if href:
                # Check if href matches our two target patterns
                if ("https://placng.org/lawsofnigeria/laws/" in href or
                    "https://placng.org/lawsofnigeria/view2.php?sn=" in href):

                    link_text = link.text.strip()

                    # If no visible text, try other attributes
                    if not link_text:
                        link_text = link.get_attribute('title') or link.get_attribute('innerText') or ''
                        link_text = link_text.strip()

                    links_data.append({
                        'url': href,
                        'link_text': link_text
                    })

        print(f"Page {page_number}: Found {len(links_data)} matching links")
        return links_data

    except Exception as e:
        print(f"Error processing page {page_number}: {e}")
        return []

def create_enhanced_csv():
    """Create CSV with url and link_text columns for all matching links"""
    all_links_data = []
    total_pages = 69

    print(f"🎯 Extracting links matching patterns:")
    print(f"  1. https://placng.org/lawsofnigeria/laws/*")
    print(f"  2. https://placng.org/lawsofnigeria/view2.php?sn=*")
    print(f"📄 Processing {total_pages} pages...")

    for page_num in tqdm(range(1, total_pages + 1), desc="Extracting links"):
        page_links = extract_links_with_text_from_page(page_num)
        all_links_data.extend(page_links)
        time.sleep(0.5)  # Respectful delay

    # Remove duplicates while preserving order
    seen_urls = set()
    unique_links_data = []

    for link in all_links_data:
        if link['url'] not in seen_urls:
            seen_urls.add(link['url'])
            unique_links_data.append(link)

    # Create DataFrame
    df = pd.DataFrame(unique_links_data)

    # Add file type classification
    df['file_type'] = df['url'].apply(
        lambda x: 'PDF' if '/laws/' in x else 'HTML'
    )

    # Reorder columns
    df = df[['url', 'link_text', 'file_type']]

    # Save to CSV
    output_filename = "nigeria_laws_urls_with_link_text.csv"
    df.to_csv(output_filename, index=False)

    print(f"\n✅ CSV created: {output_filename}")
    print(f"📊 Total unique links: {len(df)}")
    print(f"📄 PDF links (/laws/): {len(df[df['file_type'] == 'PDF'])}")
    print(f"🌐 HTML links (view2.php): {len(df[df['file_type'] == 'HTML'])}")
    print(f"📝 Links with non-empty text: {len(df[df['link_text'] != ''])}")

    # Display sample
    print(f"\n🔍 Sample data:")
    print(df.head(10).to_string(index=False))

    return df

# Run the extraction
try:
    df = create_enhanced_csv()
    print(f"\n🎉 Extraction complete! File saved as 'nigeria_laws_urls_with_link_text.csv'")
finally:
    driver.quit()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 https://cli.github.com/packages stable InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,168 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,594 

Extracting links:   0%|          | 0/69 [00:00<?, ?it/s]

Processing page 1...
Page 1: Found 8 matching links


Extracting links:   1%|▏         | 1/69 [00:02<02:28,  2.18s/it]

Processing page 2...
Page 2: Found 8 matching links


...

Processing page 68...
Page 68: Found 8 matching links


Extracting links:  99%|█████████▊| 68/69 [01:12<00:01,  1.05s/it]

Processing page 69...
Page 69: Found 2 matching links


Extracting links: 100%|██████████| 69/69 [01:13<00:00,  1.07s/it]



✅ CSV created: nigeria_laws_urls_with_link_text.csv
📊 Total unique links: 546
📄 PDF links (/laws/): 133
🌐 HTML links (view2.php): 413
📝 Links with non-empty text: 546

🔍 Sample data:
                                             url                                                                       link_text file_type
    https://placng.org/lawsofnigeria/laws/A1.pdf                                   ABUBAKAR TAFAWA BALEWA UNIVERSITY, BAUCHI ACT       PDF
https://placng.org/lawsofnigeria/view2.php?sn=14                                                         ACTS AUTHENTICATION ACT      HTML
https://placng.org/lawsofnigeria/view2.php?sn=15                                        ADMINISTRATION OF JUSTICE COMMISSION ACT      HTML
https://placng.org/lawsofnigeria/view2.php?sn=16                                     ADMINISTRATIVE STAFF COLLEGE OF NIGERIA ACT      HTML
https://placng.org/lawsofnigeria/view2.php?sn=17                                                      ADMIRALTY JURISDICT

In [4]:
pdf_dir = "/content/nigeria_laws_data_20251107_112353/pdfs"
texts = "/content/nigeria_laws_data_20251107_112353/texts"

file = open("/content/nigeria_laws_data_20251107_112353/texts/_lawsofnigeria_view2.php.txt", "r")


In [5]:
pdf_files = [os.path.join(pdf_dir, f) for f in os.listdir(pdf_dir) if f.lower().endswith('.pdf')]
print(pdf_files)


['/content/nigeria_laws_data_20251107_112353/pdfs/C13.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/NATIONALBOUNDARYCOMMISSION_ESTABLISHMENT_ACT.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/F2.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/E7.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/NigerianAgriculturalInsuranceCorporationAct.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/E16.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/INTERNATIONALCONVENTION.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/A1.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/M7.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/S4.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/R12.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/U15.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/N102.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/N1.pdf', '/content/nigeria_laws_data_20251107_112353/pdfs/F13.pdf', '/content/nigeria_

In [6]:
len(pdf_files)

133

In [37]:
# Step 1: Install dependencies
!sudo apt install tesseract-ocr -y
!pip install pytesseract pdf2image Pillow

# (Optional) Install Poppler for pdf2image
!sudo apt install poppler-utils -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 43 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 0s (383 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
d

In [38]:
from pdf2image import convert_from_path
import pytesseract
from PyPDF2 import PdfReader
from PIL import Image
import os

def is_image_pdf(pdf_path):
    """
    Quick heuristic: returns True if the PDF likely contains only images.
    """
    try:
        reader = PdfReader(pdf_path)
        # If any page has extractable text, treat as text-based
        for page in reader.pages:
            text = page.extract_text()
            if text and text.strip():
                return False
        return True
    except:
        return True  # if PyPDF2 can't read, assume image-based

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF — uses PyPDF2 for text-based, pytesseract for image-based.
    Returns a string.
    """
    if is_image_pdf(pdf_path):
        # Image-based PDF → OCR
        pages = convert_from_path(pdf_path, dpi=300)
        text = ""
        for i, page in enumerate(pages):
            page_text = pytesseract.image_to_string(page, lang="eng")
            text += f"\n\n--- Page {i+1} ---\n\n" + page_text
        return text.strip()
    else:
        # Text-based PDF → direct extraction
        reader = PdfReader(pdf_path)
        text = ""
        for i, page in enumerate(reader.pages):
            text += f"\n\n--- Page {i+1} ---\n\n" + (page.extract_text() or "")
        return text.strip()

# Process all PDFs into a list
extracted_texts = [extract_text_from_pdf(p) for p in pdf_files]

# Each element corresponds to one PDF’s extracted text
len(extracted_texts), type(extracted_texts[0])


(133, str)

show first line

In [41]:
for i, text in enumerate(extracted_texts):
    print(f"\n📘 PDF {i+1}:")
    lines = text.splitlines()
    for line in lines[2:40]:  # print first 3 lines
        print(line)
    print("-" * 80)



📘 PDF 1:
         CIVIL  AVIATION  ACT  
    ARRANGEMENT  OF SECTIONS  
      PART  I 
    Control  of Air Navigation  
SECTION 
1. Control  and Supervision  of civil aviation. 
      PART  II 
    The  Nigerian  Civil Aviation Authority  
2. Establishment  of the Nigerian Civil  Aviation Authority.   
      PART  III 
    The  Governing  Board of the Authority  
3. Membership  of the Governing  Board. 
4. Tenure  of Office. 
5. Cessation  of Office. 
6. Allowances
, expenses,  etc., of Chairman  and members.   
7. Functions  of the Board. 
      PART  IV 
     Staff  of the Authority  
8. Director‐General of the Authority.  
9. Appointment  of secretary  and other staff by the Board.  
10. Conflict of interests.  
11. Pensions;  Act No. 2, 2004. 
      PART  V 
     Financi
al Provisions  
12. Air ticket and cargo sales charge. 
13. Fund of the Authority.  
14. Expenditure  by the Authority.  
15. Power to accept gifts. 
16. Power to borrow  and invest. 
17. Exemption  from tax. 
18.

In [42]:
import re

def clean_text(text):
    # Replace non-breaking spaces with regular spaces
    text = text.replace('\xa0', ' ')
    # Replace multiple newlines with a single one
    text = re.sub(r'\n+', '\n', text)
    # Remove redundant spaces
    text = re.sub(r' +', ' ', text)
    # Strip leading/trailing whitespace
    return text.strip()

pdf_texts = [clean_text(i) for i in extracted_texts]

# Save to faiss index

In [10]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 80.8 MB/s eta 0:00:00


In [11]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [12]:
model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
!pip install langchain-text-splitters


In [43]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Configure a strong, context-preserving semantic splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1800,      # number of characters per chunk
    chunk_overlap=200,    # overlap to preserve context
    separators=["\n\n", "\n", ".", "!", "?", " ", ""]
)

semantic_chunks = []
for doc_id, text in enumerate(extracted_texts):
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        semantic_chunks.append({
            "doc_id": doc_id,
            "text": chunk.strip()
        })

print(f"Created {len(semantic_chunks)} high-quality semantic chunks.")


Created 20471 high-quality semantic chunks.


In [44]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

texts = [c["text"] for c in semantic_chunks]
embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)


Batches:   0%|          | 0/640 [00:00<?, ?it/s]

In [45]:
import faiss
import numpy as np

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print(f"✅ Stored {index.ntotal} semantic chunks in FAISS.")


✅ Stored 20471 semantic chunks in FAISS.


In [46]:
import pickle

faiss.write_index(index, "eagle.index")
with open("eagle_metadata.pkl", "wb") as f:
    pickle.dump(semantic_chunks, f)


In [47]:
query = "functions of the Chancellor of Abubakar Tafawa Balewa University"
query_emb = model.encode([query], convert_to_numpy=True)
D, I = index.search(query_emb, k=5)

print("\nTop relevant chunks:")
for idx in I[0]:
    print(semantic_chunks[idx]["text"][:400])
    print("\n" + "-"*80)



Top relevant chunks:
--- Page 1 ---

CAP. Al -
Abubakar Tafawa Balewa University, Bauchi Act

 

CHAPTER Al
ABUBAKAR TAFAWA BALEWA UNIVERSITY, BAUCHI ACT

ARRANGEMENT OF SECTIONS

SECTION
1. Incorporation and objects of the Abubakar Tafawa Balewa University, Bauchi.

Constitution and principal officers of the University. ,
Powers of the University and its exercise.
Functions of the Chancellor.
Functions of the Pro-Cha

--------------------------------------------------------------------------------
“University” means the Abubakar Tafawa Balewa University, Bauchi established
under section | of this Act.

(2) It is hereby declared that where in any provision of this Act it is laid down that
proposals are to be submitted or a recommendation is to be made by one authority to an-
other through one or more intermediate authorities, it shall be the duty of every such in-
termediate authority to for

--------------------------------------------------------------------------------
SCHEDULES

F

In [49]:
!ls -lh eagle_metadata.pkl

-rw-r--r-- 1 root root 22M Nov  7 15:43 eagle_metadata.pkl
